In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
def huber(x: np.array, k: float):
    phi = lambda elem: elem if abs(elem) < k else k * np.sign(elem)

    return np.mean([phi(elem) for elem in x])

def sigm3_rull(x: np.array):

    xmean = np.mean(x)
    xvar = np.var(x)
    is_in_3sigm = lambda xi: np.abs(xi - xmean) < 3*xvar

    clear_res = [val for val in x if is_in_3sigm(val)]

    return clear_res

def double_stage_mean(x: np.array):
    x = sigm3_rull(x)
    return np.mean(x)

In [3]:
distributions = {
    "Нормальное": stats.norm.rvs,
    "Коши": stats.cauchy.rvs,
    "Смешанное": lambda size: 0.9 * stats.norm.rvs(size=size) + 0.1 * stats.cauchy.rvs(size=size) 
}

measures = {
    "среднее": np.mean,
    "медиана": np.median,
    "Хубер": lambda x: huber(x, 1.44),
    "2-эт": double_stage_mean
}


In [4]:
def monte_karlo(N: int, sample_size: int, dist_grvs, measure):
    means = [measure(dist_grvs(size=sample_size)) for _ in range(N)]

    return np.mean(means), np.var(means)

In [5]:
def main(N : int = 10000, sample_size: int = 100):
    for dname, grvs in distributions.items():
        print(dname)
        for mname, measure in measures.items():
            mu, var = monte_karlo(N, sample_size, grvs, measure)
            print(f"\t{mname}")
            print(f"\t\tСреднее:\t{mu:.6f}")
            print(f"\t\tДисперсия:\t{var:.6f}")
        print("")


In [6]:
main()

Нормальное
	среднее
		Среднее:	0.000353
		Дисперсия:	0.009986
	медиана
		Среднее:	0.000134
		Дисперсия:	0.015156
	Хубер
		Среднее:	0.001502
		Дисперсия:	0.007392
	2-эт
		Среднее:	0.001453
		Дисперсия:	0.010368

Коши
	среднее
		Среднее:	-0.281869
		Дисперсия:	2609.232260
	медиана
		Среднее:	-0.000656
		Дисперсия:	0.025492
	Хубер
		Среднее:	0.001141
		Дисперсия:	0.011014
	2-эт
		Среднее:	0.481793
		Дисперсия:	6102.794494

Смешанное
	среднее
		Среднее:	-0.004168
		Дисперсия:	29.428458
	медиана
		Среднее:	-0.000059
		Дисперсия:	0.014893
	Хубер
		Среднее:	0.000315
		Дисперсия:	0.007580
	2-эт
		Среднее:	0.046917
		Дисперсия:	47.688622

